# Experiment with Azure Digital Tween Service
[Location](https://explorer.digitaltwins.azure.net/?tid=76a2ae5a-9f00-4f6b-95ed-5d33d77c4d61&eid=digitaltwins.api.weu.digitaltwins.azure.net)

## Set up

### Create Service Instance
Create instance as described [here](https://learn.microsoft.com/en-gb/azure/digital-twins/quickstart-azure-digital-twins-explorer)

### Define models
To initial define set of models (ontology) was used Office 365 copilot and [prompt](prompts/Define%20models.txt). Some extra tuning was required.
Models are stored in `ontology` folder.
Then models were uploaded to service.

### Init

In [1]:
import os
from dotenv import load_dotenv
from azure.digitaltwins.core import DigitalTwinsClient
from azure.identity import DefaultAzureCredential

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
digital_twin_endpoint = os.getenv("AZURE_DIGITAL_TWINS_ENDPOINT")

credential = DefaultAzureCredential()
# Create an instance of the DigitalTwinsClient
client = DigitalTwinsClient(digital_twin_endpoint, credential)

## Experiments

### Connect to ADT

In [8]:
import json

def upd_twin(twin_id, twin_data):
    # Send data to the topology
    try:
        twin_data_dict = json.loads(twin_data)
        client.upsert_digital_twin(twin_id, twin_data_dict)
        return "SUCCESS"
    except Exception as e:
        print("Error sending data to Azure Digital Twins service topology:", str(e))
        return "FAIL"
    
def upd_rel(twin_id, relation_id, relation_data):
    try:
        relation_data_dict = json.loads(relation_data)
        client.upsert_relationship(twin_id, relation_id, relation_data_dict)
        return "SUCCESS"
    except Exception as e:
        print("Error sending data to Azure Digital Twins service topology:", str(e))
        return "FAIL"

twin_id = "HUB_4"
twin_data = """
    {
        "$metadata":{
            "$model":"dtmi:com:example:ControlHub;1"
        },
        "name":"Hub_4_name",
        "location":"Kyiv",
        "status":"on",
        "$relationships":{
            "connects":{
                "$targetId":"CentralUnit",
                "$relationshipName":"connects"
            }
        }
    }
"""

twin_id_2 = "CentralUnit"
relation_id="relationship1id"
relation_data = """
{
    "$relationshipId": "relationship1id",
    "$sourceId": "CentralUnit",
    "$relationshipName": "connects",
    "$targetId": "HUB_4",
    "$metadata": {
        "$model": "dtmi:com:example:CentralUnit;1"
    }
}
"""


In [52]:
upd_twin(twin_id, twin_data)

'SUCCESS'

In [9]:
upd_rel(twin_id_2, relation_id, relation_data)

### Request data from ADT

In [ ]:
import json
def query_twin() -> str:
    client = DigitalTwinsClient(digital_twin_endpoint, credential)

    # Define your query
    query = "SELECT * FROM digitaltwins T WHERE IS_OF_MODEL(T, 'dtmi:com:example:CentralUnit;1')"

    # Execute the query
    query_result = client.query_twins(query)

    # Print the results
    #for twin in query_result:
    #    print(twin)

    return json.dumps(list(query_result), indent=4)

r = query_twin()

print(r)